In [1]:
import pandas as pd 
import talib

data_path = '../input/'
train = pd.read_pickle(data_path+'/train.pkl')
train.head()

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
0,1514764860,2,40.0,2376.5800,2399.5000,2357.1400,2374.5900,19.233005,2373.116392,-0.004218
1,1514764860,0,5.0,8.5300,8.5300,8.5300,8.5300,78.380000,8.530000,-0.014399
2,1514764860,1,229.0,13835.1940,14013.8000,13666.1100,13850.1760,31.550062,13827.062093,-0.014643
3,1514764860,5,32.0,7.6596,7.6596,7.6567,7.6576,6626.713370,7.657713,-0.013922
4,1514764860,7,5.0,25.9200,25.9200,25.8740,25.8770,121.087310,25.891363,-0.008264


In [2]:
train.groupby('Asset_ID').Close.transform(lambda x: x.rolling(window=5).mean())

0                   NaN
1                   NaN
2                   NaN
3                   NaN
4                   NaN
               ...     
24236801     156.809571
24236802    2426.988343
24236803       0.091123
24236804       0.281477
24236805     231.791100
Name: Close, Length: 24236806, dtype: float64

In [3]:
train.groupby('Asset_ID').Close.transform(lambda x: talib.SMA(x, timeperiod=5))

0                   NaN
1                   NaN
2                   NaN
3                   NaN
4                   NaN
               ...     
24236801     156.809571
24236802    2426.988343
24236803       0.091124
24236804       0.281477
24236805     231.791100
Name: Close, Length: 24236806, dtype: float64

In [4]:
import sys
sys.path.append('../src')
from utils import RunningMean


In [18]:
for i in train.Asset_ID.unique():
    a = RunningMean(WIN_SIZE=5)
    a.push(train.loc[train.Asset_ID==i, ['Close', 'Volume']].values)
    mean = a.get_mean()
    train.loc[train.Asset_ID==i, 'close_rm'], train.loc[train.Asset_ID==i, 'volume_rm'] = pd.Series(mean[: 0].reshape(-1)), pd.Series(mean[: 1].reshape(-1)) 
train

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target,close_rm,volume_rm
0,1514764860,2,40.0,2376.580000,2399.500000,2357.140000,2374.590000,1.923301e+01,2373.116392,-0.004218,NaN,2374.59
1,1514764860,0,5.0,8.530000,8.530000,8.530000,8.530000,7.838000e+01,8.530000,-0.014399,NaN,78.38
2,1514764860,1,229.0,13835.194000,14013.800000,13666.110000,13850.176000,3.155006e+01,13827.062093,-0.014643,NaN,NaN
3,1514764860,5,32.0,7.659600,7.659600,7.656700,7.657600,6.626713e+03,7.657713,-0.013922,NaN,NaN
4,1514764860,7,5.0,25.920000,25.920000,25.874000,25.877000,1.210873e+02,25.891363,-0.008264,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
24236801,1632182400,9,775.0,157.181571,157.250000,156.700000,156.943857,4.663725e+03,156.994319,NaN,NaN,NaN
24236802,1632182400,10,34.0,2437.065067,2438.000000,2430.226900,2432.907467,3.975460e+00,2434.818747,NaN,NaN,NaN
24236803,1632182400,13,380.0,0.091390,0.091527,0.091260,0.091349,2.193732e+06,0.091388,NaN,NaN,NaN
24236804,1632182400,12,177.0,0.282168,0.282438,0.281842,0.282051,1.828508e+05,0.282134,NaN,NaN,NaN


RSIめっちゃ遅かった
talib使って早くなった

In [6]:
import numpy as np

def _rsiFunc(prices, n=14):
    deltas = np.diff(prices)
    seed = deltas[:n+1]
    up = seed[seed>=0].sum()/n
    down = -seed[seed<0].sum()/n
    rs = up/down
    rsi = np.zeros_like(prices)
    rsi[:n] = 100. - 100./(1.+rs)

    for i in range(n, len(prices)):
        delta = deltas[i-1] # cause the diff is 1 shorter

        if delta>0:
            upval = delta
            downval = 0.
        else:
            upval = 0.
            downval = -delta

        up = (up*(n-1) + upval)/n
        down = (down*(n-1) + downval)/n

        rs = up/down
        rsi[i] = 100. - 100./(1.+rs)

    return rsi



In [7]:
train.groupby('Asset_ID').Close.transform(lambda x: _rsiFunc(x.values, 5))

0            0.000000
1            9.675481
2            0.000000
3            0.000000
4           86.370262
              ...    
24236801    49.859647
24236802    57.611476
24236803    58.297421
24236804    55.637040
24236805    55.028428
Name: Close, Length: 24236806, dtype: float64

In [8]:
import talib

In [9]:
train.groupby('Asset_ID').Close.transform(lambda x: talib.RSI(x, 5))

0                 NaN
1                 NaN
2                 NaN
3                 NaN
4                 NaN
              ...    
24236801    49.859647
24236802    57.611476
24236803    58.297421
24236804    55.637040
24236805    55.028428
Name: Close, Length: 24236806, dtype: float64